Initializing

In [ ]:
!git clone https://github.com/qiuqiangkong/audioset_tagging_cnn.git
%cd audioset_tagging_cnn

Install required packages

In [ ]:
!pip install torch
!pip install torchlibrosa

Download the dataset

In [ ]:
%env DATASET_DIR=./datasets/audioset201906
%env WORKSPACE=./workspaces/audioset_tagging
!mkdir -p $DATASET_DIR"/metadata"
!wget -O $DATASET_DIR"/metadata/eval_segments.csv" http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/eval_segments.csv
!wget -O $DATASET_DIR"/metadata/balanced_train_segments.csv" http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/balanced_train_segments.csv
!wget -O $DATASET_DIR"/metadata/class_labels_indices.csv" http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/class_labels_indices.csv
!wget -O $DATASET_DIR"/metadata/qa_true_counts.csv" http://storage.googleapis.com/us_audioset/youtube_corpus/v1/qa/qa_true_counts.csv

Download and index balanced training dataset


In [ ]:
!python3 utils/dataset.py download_wavs --csv_path=$DATASET_DIR"/metadata/eval_segments.csv" --audios_dir=$DATASET_DIR"/audios/eval_segments"
!python3 utils/dataset.py pack_waveforms_to_hdf5 --audios_dir="/content/audioset_tagging_cnn/datasets/audioset201906/audios/eval_segments" --csv_path="/content/audioset_tagging_cnn/datasets/audioset201906/metadata/eval_segments.csv" --waveforms_hdf5_path=$WORKSPACE"/hdf5s/waveforms/eval.h5" --mini_data
!python3 utils/create_indexes.py create_indexes --waveforms_hdf5_path=$WORKSPACE"/hdf5s/waveforms/eval.h5" --indexes_hdf5_path=$WORKSPACE"/hdf5s/indexes/eval.h5"

In [ ]:
!python3 utils/dataset.py download_wavs --csv_path=$DATASET_DIR"/metadata/balanced_train_segments.csv" --audios_dir=$DATASET_DIR"/audios/balanced_train_segments"
!python3 utils/dataset.py pack_waveforms_to_hdf5 --audios_dir="/content/audioset_tagging_cnn/datasets/audioset201906/audios/balanced_train_segments" --csv_path="/content/audioset_tagging_cnn/datasets/audioset201906/metadata/balanced_train_segments.csv" --waveforms_hdf5_path=$WORKSPACE"/hdf5s/waveforms/balanced_train.h5"
!python3 utils/create_indexes.py create_indexes --waveforms_hdf5_path=$WORKSPACE"/hdf5s/waveforms/balanced_train.h5" --indexes_hdf5_path=$WORKSPACE"/hdf5s/indexes/balanced_train.h5"

Train the model using balanced training dataset

In [ ]:
%env WORKSPACE=./workspaces/audioset_tagging
%env CUDA_VISIBLE_DEVICES=0 
!python3 pytorch/main.py train --workspace=$WORKSPACE --data_type='balanced_train' --window_size=1024 --hop_size=320 --mel_bins=64 --fmin=50 --fmax=14000 --model_type='Cnn14' --loss_type='clip_bce' --balanced='balanced' --augmentation='mixup' --batch_size=32 --learning_rate=1e-3 --resume_iteration=0 --early_stop=1000000 --cuda